Members -

Aditya Kothari - I028

In [8]:
#Importing necessary libraries and modules
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dropout, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf

In [3]:
#Reading text data from file
file = "/content/text.txt"
with open(file, "r") as f:
    mytext = f.read()

In [4]:
#Tokenizing the text data and determine the vocabulary size
mytokenizer = Tokenizer()
mytokenizer.fit_on_texts([mytext])
total_words = len(mytokenizer.word_index) + 1

Stop word removal, lemmatization and stemming weren't used to keep the context of the data and words like to, the etc. would also be used for prediction.

In [5]:
#Generating input sequences and their corresponding outputs
my_input_sequences = []
for line in mytext.split('\n'):
    token_list = mytokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        my_n_gram_sequence = token_list[:i+1]
        my_input_sequences.append(my_n_gram_sequence)

In [6]:
#Padding the input sequences to ensure uniform length
max_sequence_len = max([len(seq) for seq in my_input_sequences])
input_sequences = np.array(pad_sequences(my_input_sequences, maxlen=max_sequence_len, padding='pre'))

In [9]:
#Preparing input and output data for the model
X = input_sequences[:, :-1]
y = input_sequences[:, -1]
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [27]:
#Defining the LSTM model architecture
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(200, return_sequences=True)))
model.add(LSTM(378))
model.add(Dropout(0.2))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 82, 100)           59900     
                                                                 
 bidirectional_10 (Bidirect  (None, 82, 400)           481600    
 ional)                                                          
                                                                 
 lstm_21 (LSTM)              (None, 378)               1177848   
                                                                 
 dropout_10 (Dropout)        (None, 378)               0         
                                                                 
 dense_9 (Dense)             (None, 599)               227021    
                                                                 
Total params: 1946369 (7.42 MB)
Trainable params: 1946369 (7.42 MB)
Non-trainable params: 0 (0.00 Byte)
_______________

In [28]:
#Compiling the model with appropriate loss function and optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
#Training the model with the prepared data
model.fit(X, y, epochs=50, verbose=1)

Epoch 1/50
43/43 [==============================] - 38s 110ms/step - loss: 6.1639 - accuracy: 0.0343
Epoch 2/50
43/43 [==============================] - 4s 90ms/step - loss: 5.8039 - accuracy: 0.0547
Epoch 3/50
43/43 [==============================] - 3s 69ms/step - loss: 5.7331 - accuracy: 0.0576
Epoch 4/50
43/43 [==============================] - 3s 59ms/step - loss: 5.6013 - accuracy: 0.0576
Epoch 5/50
43/43 [==============================] - 2s 49ms/step - loss: 5.4408 - accuracy: 0.0554
Epoch 6/50
43/43 [==============================] - 2s 35ms/step - loss: 5.3296 - accuracy: 0.0693
Epoch 7/50
43/43 [==============================] - 1s 30ms/step - loss: 5.2080 - accuracy: 0.0780
Epoch 8/50
43/43 [==============================] - 2s 44ms/step - loss: 5.0366 - accuracy: 0.0926
Epoch 9/50
43/43 [==============================] - 1s 34ms/step - loss: 4.8411 - accuracy: 0.0985
Epoch 10/50
43/43 [==============================] - 1s 25ms/step - loss: 4.6361 - accuracy: 0.1189
Epoch 1

98.4% accuracy achieved

It can be improved by increasing the number of epochs but that would be a result of overfitting.

98.98% accuracy was achieved by runnning 100 epochs but that led to overfitting, therefore the no. of epochs were reduced.

In [26]:
#Defining the initial input text for word prediction
input_text = input("Enter the word: ")

#Setting the number of words to predict
predict_next_words = 4

#Generating a sequence of predicted words based on the trained model
for _ in range(predict_next_words):
    token_list = mytokenizer.texts_to_sequences([input_text])[0]
    print(token_list)
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in mytokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    input_text += " " + output_word

#Printing the generated text based on the predicted words
print(input_text)

Enter the word: India
[6]
1/1 [==============================] - 0s 23ms/step
[6, 5]
1/1 [==============================] - 0s 25ms/step
[6, 5, 1]
1/1 [==============================] - 0s 22ms/step
[6, 5, 1, 9]
1/1 [==============================] - 0s 21ms/step
India and the us have
